# PART1

## Json import to openSearch

In [ ]:
import createRecipesMap as crm

crm.CreateRecipesMap("../jsonData/recipe_queries_results.json", '../jsonData/recipesMap.json').createMap()

## Login to openSearch

In [ ]:
import loginOpenSearch as lop

login = lop.LoginOpenSearch("../jsonData/openSearchConfig.json").login()
    
client = login[0]
index_name = login[1]

## Create index structure

In [ ]:
import createIndexes as ci

ci.CreateIndexes(client, index_name).createIndexStructure()

## Choose the tokenizer and model that will be used in the embeddings

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")
model = AutoModel.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")

In [ ]:
import computeEmbeddings as ce

ce.ComputeEmbeddings("../jsonData/recipesMap.json","../jsonData/recipesEmbeddingsMap.json", tokenizer, model).createMap()

In [ ]:
import indexRecipes as ir

ir.IndexRecipes("../jsonData/recipesMap.json", "../jsonData/recipesEmbeddingsMap.json", client, index_name).indexRecipes()


In [ ]:
import search as s

s.Search(client, index_name, tokenizer, model).queryOpenSearch('Holiday Salad', 5,None, None, ["salads"], ["lupine"], None)

## Testing

In [ ]:
import searchJson as sj
metrics = sj.SearchJson(client, index_name, tokenizer, model)
metrics.searchJson("../jsonData/cleanAnnotations.json")

In [ ]:
delete this sentence to run
if client.indices.exists(index=index_name):
    # Delete the index.
    response = client.indices.delete(
        index = index_name,
        timeout = "600s"
    )
    print('\nDeleting index:')
    print(response)

# PART2

## Compute the step similarity matrix

### Compute the steps json files

In [ ]:
import stepsCalculator as sc

sc.CreateStepsMaps("../jsonData/recipesMap.json", '../jsonData/stepsImage.json', '../jsonData/stepsNoImage.json').createMaps()


### Compute the embeddings

In [ ]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"

device

In [ ]:
import computeImagesEmbeddings as cie

cie.ComputeImagesEmbeddings("../jsonData/stepsImage.json","../jsonData/stepsNoImage.json","../jsonData/stepsImageEmbeddingsMap.json","../jsonData/stepsNoImageEmbeddingsMap.json","images/").createMap()

### Compute the similarity matrix

In [ ]:
import computeMatrix as cm

imageSimilarityMatrix = cm.ComputeMatrix("../jsonData/stepsImageEmbeddingsMap.json", "../jsonData/stepsNoImageEmbeddingsMap.json").createMatrix()
print(imageSimilarityMatrix)
print(len(imageSimilarityMatrix[0]))

In [ ]:
import getImage as gi

gi.GetImage(imageSimilarityMatrix, "../jsonData/stepsImage.json", "../jsonData/stepsNoImage.json", "images/","../jsonData/recipesMap.json").getImageFunc()

In [ ]:
#import computeClipOutput as cco

#image = cco.ComputeClipOutput(imageSimilarityMatrix, "../jsonData/stepsImage.json", "../jsonData/stepsNoImage.json", "images/","../jsonData/recipesMap.json").getImage()

# PART3


In [1]:
import os
import numpy as np
import transformers
import json
import loginOpenSearch as lop
import search as s
import extractiveQA as eq

import random

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    AutoModel,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)

model_finetuned = '/user/data/public/twiz-intent-model'
with open(os.path.join(model_finetuned + '/all_intents.json'), 'r') as all_intents_json:
    all_intents = json.load(all_intents_json) # contains the written out names of intents. also implicitly

tokenizer_name = 'roberta-base' # try 'bert-base-uncased', 'bert-base-cased', 'bert-large-uncased'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name) # loads a tokenizer

model = AutoModelForSequenceClassification.from_pretrained(model_finetuned, num_labels=len(all_intents)) # Loads the BERT model weights

login = lop.LoginOpenSearch("../jsonData/openSearchConfig.json").login()
openSearchTokenizer = AutoTokenizer.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")
openSearchModel = AutoModel.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")

searchEngine = s.Search(login[0], login[1], openSearchTokenizer, openSearchModel)
qaExtractor = eq.ExtractiveQA()



----------------------------------------------------------------------------------- INDEX SETTINGS
{'user209': {'settings': {'index': {'creation_date': '1649609449190',
                                    'knn': 'true',
                                    'number_of_replicas': '0',
                                    'number_of_shards': '4',
                                    'provided_name': 'user209',
                                    'refresh_interval': '1s',
                                    'uuid': 'zYvHS7n3SS6ejuPIp7Ikhw',
                                    'version': {'created': '135238227'}}}}}

----------------------------------------------------------------------------------- INDEX MAPPINGS
{'user209': {'mappings': {'properties': {'description': {'similarity': 'BM25',
                                                         'type': 'text'},
                                         'ingredients': {'type': 'keyword'},
                                         'negative_Ke

In [3]:
import stateMachine
turing = stateMachine.StateMachine(tokenizer,model, all_intents,searchEngine, qaExtractor)

#response = 'hello, i would like to cook spaghetti'
response = 'I\'m in the mood for some carbonara.'
intent = turing.getIntent(response)
turing.setUserResponse(response)
print(intent)
getattr(turing, intent)()

response = 'i would like my recipe to contain some apples and some onions but not bananas'
intent = turing.getIntent(response)
turing.setUserResponse(response)
print(intent)
getattr(turing, intent)()

response = 'i would like my recipe to contain some apples and some onions but not bananas' 
intent = turing.getIntent(response)
turing.setUserResponse(response)
print(intent)
getattr(turing, intent)()


response = 'yes i would like my recipe to be vegetarian or gluten-free please' 
intent = turing.getIntent(response)
turing.setUserResponse(response)
print(intent)
getattr(turing, intent)()


#response = 'yes i would like the recipe to be under 60 minutes' 
response = 'yes i would like the recipe to be under sixty minutes' 
intent = turing.getIntent(response)
turing.setUserResponse(response)
print(intent)
getattr(turing, intent)()

response = 'the number 5 pls' 
intent = turing.getIntent(response)
turing.setUserResponse(response)
print(intent)
getattr(turing, intent)()

# response = 'yes' 
# intent = turing.getIntent(response)
# turing.setUserResponse(response)
# print(intent)
# getattr(turing, intent)()

# response = 'yes' 
# intent = turing.getIntent(response)
# turing.setUserResponse(response)
# print(intent)
# getattr(turing, intent)()



Hello! I'm a cooking assistant and I'm here to help you cook anything you want. What would you like to prepare today?
IdentifyProcessIntent
carbonara
Ok! I'll now ask you some questions to find the carbonara recipe you're looking for!
Are there any desired ingredients you'd like the recipe to have? If so, could you enumerate them?
IngredientsConfirmationIntent
['apples', 'onions']
Ok, I'll take apples, onions in consideration.
Are there any ingredients you really don't want in the recipe? If so, could you also enumerate them?
IngredientsConfirmationIntent
['bananas']
Got it, I'll find recipes which don't contain any bananas.
Should the recipe follow any dietary requirements?
For example, does it need to be vegan or gluten-free?
AMAZON.YesIntent
vegetarian or gluten-free
keywords: ['vegetarian', 'gluten-free']
keywordsNegative: ['gluten']
keywordsPositive: ['vegetarian']
Ok, I'll prioritize recipes that have vegetarian, gluten-free requirements.
Last question! Are there any time restric

True

In [1]:
import stateMachine
turing = stateMachine.StateMachine(tokenizer,model, all_intents, searchEngine, qaExtractor)
while(turing.state != 'terminar'):

    response = input("USER:")
    intent = turing.getIntent(response)
    turing.setUserResponse(response)
    try: 
        getattr(turing, intent)() 
    except Exception as e:
        print ('pls refrase')
        continue

        

NameError: name 'tokenizer' is not defined

In [4]:
qaExtractor.extractAnswer("What do you want to cook?", "I'm in the mood for some bacon and eggs.")


{'score': 0.7682953476905823,
 'start': 25,
 'end': 39,
 'answer': 'bacon and eggs'}

In [ ]:
from IPython.display import Image, HTML, display

with open('../jsonData/recipesMapWithImages.json', "r") as read_file:
    recipesMap = json.load(read_file)

def displayResults(title, img, totalTime, rating):
    display(HTML(f"""
        <div class="row" style="display: flex; align-items: center; border-style: double;">
            <div class="column">
                <img src={img} style="width:200px; margin-right:20px"/>
            </div>
            <div class="column">
                <div class="row"><b>{title}</b> </div>
                <div class="row">Preparation time: {totalTime}</div>
                <div class="row">Rating: {rating}</div>
            </div>
        </div>
    """))

myjson = searchEngine.queryOpenSearch('Holiday Salad', 10,None, None, ["salads"], ["lupine"], None)

recipesarray = [recipe['fields']['recipeId'][0] for recipe in myjson['hits']['hits']]


for i in recipesarray:
    title = recipesMap[i]['recipe']['displayName']
    img = recipesMap[i]['recipe']['images'][0]['url']
    totalTime = str(recipesMap[i]['recipe']['totalTimeMinutes']) + ' minutes' if recipesMap[i]['recipe']['totalTimeMinutes'] != None else '-'
    rating = str(recipesMap[i]['rating']['ratingValue']) + '/5' if recipesMap[i]['rating'] != None else '-'
    displayResults(title,img,totalTime,rating)    



In [29]:
import spacy as spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")

sentence = 'vegetarian or gluten-free'
idk = nlp(sentence)
output = []
for token in idk:
    if(token.is_alpha):
        toRemove = token.text
    print
    output.append(token)

output

[vegetarian, or, gluten, -, free]

In [37]:
import spacy as spacy
from spacy import displacy


def checkNegative(keyword):
    return "free" in keyword or "no" in keyword

def cleanNegativeWord(keyword):
    keyword = keyword.replace("free","").strip()
    keyword = keyword.replace("no ","").strip()
    return keyword

nlp = spacy.load("en_core_web_sm")

sentence = 'vegetarian or gluten-free or fish-free and nice'
array = sentence.split()
keywordsPositive = []
keywordsNegative = []

doc = nlp(sentence)
for token in doc:
    if(token.is_stop ):
        array.remove(token.text)

print(array)

for keyword in array:
    keywordDoc = nlp(keyword)
    keywordString = ' '.join([token.lemma_ for token in keywordDoc if not token.is_stop and token.is_alpha])
    print(keywordString)
    if checkNegative(keyword):
        keywordCleaned = cleanNegativeWord(keywordString)
        keywordsNegative.append(keywordCleaned)
    else:
        keywordsPositive.append(keywordString)



print(keywordsNegative)

print(keywordsPositive)

['vegetarian', 'gluten-free', 'fish-free', 'nice']
vegetarian
gluten free
fish free
nice
['gluten', 'fish']
['vegetarian', 'nice']


In [11]:
def text2int(textnum, numwords={}):
    if not numwords:
      units = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
        "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
        "sixteen", "seventeen", "eighteen", "nineteen",
      ]

      tens = ["", "", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]

      scales = ["hundred", "thousand", "million", "billion", "trillion"]

      numwords["and"] = (1, 0)
      for idx, word in enumerate(units):    numwords[word] = (1, idx)
      for idx, word in enumerate(tens):     numwords[word] = (1, idx * 10)
      for idx, word in enumerate(scales):   numwords[word] = (10 ** (idx * 3 or 2), 0)

    current = result = 0
    for word in textnum.split():
        if word not in numwords:
          raise Exception("Illegal word: " + word)

        scale, increment = numwords[word]
        current = current * scale + increment
        if scale > 100:
            result += current
            current = 0

    return result + current



In [13]:
text2int("sixty")

60